In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn import preprocessing
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras import metrics
from sklearn.model_selection import train_test_split
from skopt import BayesSearchCV
from keras.wrappers.scikit_learn import KerasRegressor

Using TensorFlow backend.
e:\envs\python36env\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
e:\envs\python36env\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
e:\envs\python36env\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
e:\envs\python36env\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning:

In [2]:
data_csv = pd.read_csv('EURUSD_data_D1.csv')

In [3]:
data_csv.drop(['Unnamed: 0'], axis=1, inplace=True)

In [4]:
print(data_csv)

        close
0     1.16680
1     1.17140
2     1.15990
3     1.14230
4     1.14040
...       ...
6005  1.08319
6006  1.08386
6007  1.08060
6008  1.08469
6009  1.08173

[6010 rows x 1 columns]


In [5]:
# #Max-Min标准化
# #建立MinMaxScaler对象
# scaler = preprocessing.StandardScaler()
# # 标准化处理

# data = scaler.fit_transform(data_csv)
data = np.array(data_csv)
data

array([[1.1668 ],
       [1.1714 ],
       [1.1599 ],
       ...,
       [1.0806 ],
       [1.08469],
       [1.08173]])

In [6]:
data_y = data[-6000:,:]
print(data_y.shape)

(6000, 1)


In [7]:
data_x = []
for r in range(6000):
    for c in range(10):
        item = []
        item.append(data[r+c, 0])
        data_x.append(item)
data_x = np.array(data_x)
print(data_x)
print(data_x.shape)

[[1.1668 ]
 [1.1714 ]
 [1.1599 ]
 ...
 [1.08386]
 [1.0806 ]
 [1.08469]]
(60000, 1)


In [8]:
data_x=data_x.reshape(6000,10,1)

In [9]:
data_x.shape

(6000, 10, 1)

In [10]:
data_x_train = data_x[:,:, :]
data_y_train = data_y[:,:]

data_x_test = data_x[-1000:,:, :]
data_y_test = data_y[-1000:,:]

In [11]:
print(data_x_train.shape)
print(data_y_train.shape)
print(data_x_test.shape)
print(data_y_test.shape)

(6000, 10, 1)
(6000, 1)
(1000, 10, 1)
(1000, 1)


In [13]:
def create_model(output_dim1, num_layer, output_dim2):
    model = Sequential()

    model.add(LSTM(output_dim=output_dim1, 
                input_shape=(10, 1),
                activation='relu',
                return_sequences=True))
    for i in range(num_layer):
        model.add(LSTM(output_dim=output_dim2,
                        activation='tanh',
                        return_sequences=True))
#     model.add(Dropout(0.1))
    model.add(LSTM(output_dim=output_dim2,
                        activation='tanh'))
    model.add(Dense(1))

    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
#     model.summary()
    return model

In [13]:
# #采用暴力搜索，所有参数进行组合，然后选择最好的参数
# tuned_parameters=[{'output_dim1':Integer(28, 128),
#                    'num_layer':Integer(3,12),
#                    'output_dim2':Integer(28, 128)}]
# # 'output_dim1':Integer(28, 128),
# #             'num_layer':Integer(3,12),
# #             'output_dim2':Integer(28, 128)

In [14]:
# train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, train_size=0.80, random_state=0)

In [15]:
# #采用随机搜索，给参数一个范围，然后系统随机选择参数，进行检验，然后选择最好的
# tuned_parameters={'output_dim1':[28, 64, 96, 128],
#                    'num_layer':[4,5,6],
#                    'output_dim2':[28, 64, 96, 128]}

# lstm=RandomizedSearchCV(model,tuned_parameters,scoring='neg_mean_squared_error',n_iter=2)
 
# lstm.fit(data_x_train,data_y_train)
# print('best parameters:',lstm.best_estimator_)
# # print(classification_report(test_y,clf.predict(test_x)))
# # print(metrics.confusion_matrix(test_y,clf.predict(test_x)))

In [14]:
model=KerasRegressor(build_fn=create_model)

In [15]:
bayes_cv_tuner = BayesSearchCV(estimator = model,
                               search_spaces = { 'output_dim1':[35,86,128],
                                                'num_layer':[3,4],
                                                'output_dim2':[35,86,128]},
                               scoring = 'neg_mean_squared_error',
                               cv = 5,
                               n_iter = 2,   
                               verbose = 1,
                               refit = True,
                               random_state = 1885,
                               return_train_score = True)


In [16]:
def status_print(optim_result):
    """bayes参数优化过程中．．．"""

    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    

    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest MSE: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 5),
        bayes_cv_tuner.best_params_
    ))

    # Save all model results
    lstm_name = bayes_cv_tuner.estimator.__class__.__name__
    all_models.to_csv(lstm_name+"_cv_results.csv")

In [17]:
bayes_cv_tuner.fit(data_x,data_y, callback=status_print)
# bayes_cv_tuner.fit(data_x,data_y)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Instructions for updating:
Colocations handled automatically by placer.


e:\envs\python36env\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(10, 1), activation="relu", return_sequences=True, units=86)`
  import sys
e:\envs\python36env\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(activation="tanh", return_sequences=True, units=86)`
  # This is added back by InteractiveShellApp.init_path()
e:\envs\python36env\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(activation="tanh", units=86)`
  


Instructions for updating:
Use tf.cast instead.
Epoch 1/1
4800/4800 [==============================] - 24s 5ms/step - loss: 0.0831 - mean_absolute_error: 0.1546


...

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 10.5min finished
e:\envs\python36env\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(10, 1), activation="relu", return_sequences=True, units=86)`
  import sys


Model #2
Best MSE: -0.00449
Best params: OrderedDict([('num_layer', 4), ('output_dim1', 86), ('output_dim2', 128)])



e:\envs\python36env\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(activation="tanh", return_sequences=True, units=128)`
  # This is added back by InteractiveShellApp.init_path()
e:\envs\python36env\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(activation="tanh", units=128)`
  


Epoch 1/1
6000/6000 [==============================] - 149s 25ms/step - loss: 0.0584 - mean_absolute_error: 0.1223


BayesSearchCV(cv=5, error_score='raise',
              estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x000001FE0694D0B8>,
              fit_params=None, iid=True, n_iter=2, n_jobs=1, n_points=1,
              optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=1885,
              refit=True, return_train_score=True,
              scoring='neg_mean_squared_error',
              search_spaces={'num_layer': [3, 4], 'output_dim1': [35, 86, 128],
                             'output_dim2': [35, 86, 128]},
              verbose=1)

In [18]:
bayes_cv_tuner.best_params_

OrderedDict([('num_layer', 4), ('output_dim1', 86), ('output_dim2', 128)])

In [21]:
model = create_model(86, 4, 128)

e:\envs\python36env\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(10, 1), activation="relu", return_sequences=True, units=86)`
  import sys
e:\envs\python36env\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(activation="tanh", return_sequences=True, units=128)`
  # This is added back by InteractiveShellApp.init_path()
e:\envs\python36env\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(activation="tanh", units=128)`
  


In [22]:
model.fit(data_x_train, data_y_train, epochs=20, batch_size=2000, verbose=1)

Epoch 1/20
6000/6000 [==============================] - 50s 8ms/step - loss: 1.3906 - mean_absolute_error: 1.1681
Epoch 2/20
6000/6000 [==============================] - 23s 4ms/step - loss: 1.0069 - mean_absolute_error: 0.9876
Epoch 3/20
6000/6000 [==============================] - 23s 4ms/step - loss: 0.2562 - mean_absolute_error: 0.4637
Epoch 4/20
6000/6000 [==============================] - 23s 4ms/step - loss: 0.1204 - mean_absolute_error: 0.2867
Epoch 5/20
6000/6000 [==============================] - 23s 4ms/step - loss: 0.1033 - mean_absolute_error: 0.2912
Epoch 6/20
6000/6000 [==============================] - 24s 4ms/step - loss: 0.1007 - mean_absolute_error: 0.2867
Epoch 7/20
6000/6000 [==============================] - 24s 4ms/step - loss: 0.0244 - mean_absolute_error: 0.1275
Epoch 8/20
6000/6000 [==============================] - 24s 4ms/step - loss: 0.0613 - mean_absolute_error: 0.2210
Epoch 9/20
6000/6000 [==============================] - 24s 4ms/step - loss: 0.0206 - me

In [23]:
# 评估模型,不输出预测结果
loss,mae= model.evaluate(data_x_test,data_y_test)
print('\ntest loss','{:.5f}'.format(loss))
print('mae','{:.5f}'.format(mae))

1000/1000 [==============================] - 17s 17ms/step

test loss 0.00228
mae 0.04080


In [24]:
y_pred = model.predict(data_x_test,batch_size = 1)

In [ ]:
# y_pred[-10:]

In [ ]:
# data_y_test[-10:,0]

In [ ]:
# data_x_test[-10:,0,-1]

In [ ]:
# t = []
# for i in range(1000):
#     if data_y_test[:,0][i]>data_x_test[:,0,-1][i]:
#         t.append(1)
#     else:
#         t.append(0)
# len(t)

In [ ]:
# p = []
# for i in range(1000):
#     if y_pred[:,0][i]>data_x_test[:,0,-1][i]:
#         p.append(1)
#     else:
#         p.append(0)
# len(p)

In [ ]:
# n = 0
# for i in range(1000):
#     if t[i]==p[i]:
#         n=n+1
# n/1000

In [ ]:
# model.save('model.h5')#保存名为model的h5文件到程序所在目录

In [ ]:
# data_x_test.shape

In [ ]:
# model.predict(data_x_test,batch_size = 1)

In [ ]:
# x_pred = [1.08244, 1.0776, 1.08229, 1.08564, 1.08607, 1.08739, 1.08386, 1.09066, 1.09798, 1.0912]
# 

In [ ]:
# x_pred = np.array(x_pred).reshape(1,10,1)

In [ ]:
# x_pred.shape

In [ ]:
# model.predict(x_pred,batch_size = 1).tolist()[0][0]

In [ ]:
# y_pred.shape

In [ ]:
# y_pred

In [25]:
y_t = data_y_test.reshape(1000)

In [26]:
y_p = y_pred.reshape(1000)

In [27]:
 df_test_output = pd.DataFrame(columns=['真实值', '预测值'])

In [28]:
df_test_output['真实值'] = y_t

In [29]:
df_test_output['预测值'] = y_p

In [30]:
df_test_output

,真实值,预测值
0,1.10989,1.162232
1,1.10601,1.161124
2,1.10497,1.158916
3,1.10568,1.158885
4,1.10604,1.159496
...,...,...
995,1.08319,1.152479
996,1.08386,1.153115
997,1.08060,1.153483
998,1.08469,1.153911


In [ ]:
# df_test_output.to_csv('EURUSD_test_output.csv',encoding='utf_8_sig')